# 1.LLM Architecture 

1.GPT类型的大模型包括一下几个部分：
- token and positional embeddings
- dropout
- 一系列transformer blocks
- 一个layer normalization
- 一个输出的linear layer(输出最终的logits) 

Q: PDF书中的 Figure 4.4中说GPT model输出的vector的维度与input token embedding 维度是一样的，但是在 section 4.1的最后，对于输出的logits的介绍中发现，输出的logits中每个token的维度是vocab_size的大小，这就和Figure 4.4中不一致了，后续需要确认一下。

# 2.使用Layer Normalization归一化激活值

1.使用Layer Normalization的原因<br>
训练层数非常多的神经网络会出现梯度消失(vanishing)或爆炸(exploding)的问题，使得模型的训练会十分不稳定(unstable)，也就是参数的更新会很困难。

2.layer normalization<br>
（1）主要思想<br>
将神经网络中某一层输出的激活值调整为均值为0，方差为1（也叫做单位方差，unit variance）。<br>
（2）作用<br>
加速模型的收敛，并且保证模型训练过程的稳定。

3.layer normalization和batch normalization的区别<br>
（1）归一化的维度不同<br>
batch normalization是对batch维度进行归一化，而layer normalization是对特征维度进行归一化。

（2）使用情况不同<br>
batch normalization因为是对batch维度进行归一化，而LLM需要大量的计算资源，因此batch size通常会因为有限的计算资源而设置得非常小，进而就会影响batch normalization的效果。<br>
但是layer normalization是对特征维度进行归一化，与batch size无关，就不会受到有限计算资源的影响，更加灵活和稳定，特别适用于分布式训练或者计算资源十分有限的场景。

# 3.带GELU激活函数的前向传播网络FFN

1.LLM中常用的激活函数
除了ReLU激活函数以外，在LLM常用的还有<br>
（1）GELU(Gaussian Error Linear Unit)：高斯误差线性单元。公式为：<br>
$GELU(x) =\Phi(x)$, $\Phi$是标准高斯分布的累积分布函数。 优势是：<br>
在训练过程中接收negative输入的神经元对学习过程仍然有贡献，虽然贡献要比接收positive输入的神经元小得多。

（2）SwiGLU(Sigmoid-Weighted Linear Unit)：Sigmoid加权线性单元.公式为：
<div align="center" style="height:500;width:500;">
    <img src="./images/SwiGLU激活函数的数学表达式.jpg">
</div>

2.Feed Forward Network的作用
加强模型从数据中学习和泛化的能力。并且允许模型探索更加丰富的表示空间，因为会先对特征进行expanding，然后再contraction。